# imports

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)




log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')

log()

# qlang





In [ ]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, _type, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension



# if 'cards' not in globals():
#     cards = pd.read_csv('data/cards.csv')
# cards.q('toughness ´r >2 & <5')
# cards.qi()


# df = qp.get_df()
# df.q(
#     r"""
#     # id ´r ?1  ´n test
#     # name ´m ~ x.upper()
#     # is any ´r is any

#     date of birth / age

#     """,
#     diff=None,
#     inplace=False,
#     verbosity=4,
#     )

# df = qp.get_df()
# df.qi()






## value mask

In [ ]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension



# df = qp.get_df()
# df.qi()



# diff

In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, qpDict






df_new, df_old = get_dfs()


df_new, df_old = get_dfs()

print('df_new:')
display(df_new)

print('df_old:')
display(df_old)

print('mode=new:')
display(qp.diff(df_new, df_old, mode='new'))

print('mode=new+:')
display(qp.diff(df_new, df_old, mode='new+'))

print('mode=old:')
display(qp.diff(df_new, df_old, mode='old'))

print('mode=mix:')
display(qp.diff(df_new, df_old, mode='mix'))


In [ ]:

import pandas as pd
import numpy as np
import copy
from qplib.pd_util import _format_df
from qplib.util import log, GREEN, ORANGE, RED, GREEN_LIGHT, ORANGE_LIGHT, RED_LIGHT


 

new = 'archive/NET_BM Study_01_export_2024-04-11.xlsx'
old = 'archive/NET_BM Study_01_export_2024-03-15.xlsx'
df1, df2 = qp.get_dfs()


# diff(new, old, mode='mix', returns='test.xlsx')


# pd_util

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.util import log, qpDict
from qplib.types import _date, _na









# types

In [ ]:
import pandas as pd
import numpy as np
import re



# "bashlike" wrappers

# tests

In [ ]:

#run tests in folder "tests" using pytest and create a test report
!pytest tests --html=tests/test_report.html


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pytest
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


def get_df_simple():
    df = pd.DataFrame({
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    return df

def get_df_simple_tagged():
    df = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    df.index = [0, 1, 2]
    return df


def get_df():
    df = pd.DataFrame({
        'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
        'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
        'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
        'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
        'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
        'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
        'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
        'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
        'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
        'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
        'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
        'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
        })
    return df


def get_df_tagged():
    df1 = get_df()
    df2 = pd.DataFrame('', index=df1.index, columns=['meta', *df1.columns])
    df2.iloc[:, 1:] = df1.loc[:, :]
    return df2


df = qp.get_df()
display(df)

  




# df = get_df()
# @pytest.mark.parametrize("code, expected_cols", [
#     ('ID', ['ID']),
#     ])

# def test(code, expected_cols):
#     result = df.q(code)
#     expected = df.loc[:, expected_cols]
#     assert result.equals(expected), qp.diff(result, expected, returns='str')


vals = [
    ('´n', '', ['new1']),
    ]

for val in vals:
    code, content, cols = val
    df1 = get_df()
    result = df1.q('´s verbosity=3' + code)

    df2 = get_df()
    df2['new1'] = content
    expected = df2.loc[:, cols]

    assert result.equals(expected), f'{code}:\n{qp.diff(result, expected, returns='str')}'
  


# temp

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


df = qp.get_df()

# df.qi()

df.q(
    r"""
    ´n @name ´h name a

    name a  ´r ?j ´v a

    #is any ´r is any

    #is any  ´r is any
    """
    )